In [1]:
# Core
import numpy as np
import pandas as pd
from scipy.interpolate import CubicSpline

# Astronomy - astropy
import astropy
from astropy.units import deg, au, km, meter, day, minute, second
from astropy.coordinates import SkyCoord, ICRS, GCRS, BarycentricMeanEcliptic, HeliocentricMeanEcliptic, EarthLocation
# Astronomy - skyfield
import skyfield
from skyfield.api import Loader
skyfield_load = Loader('../data/skyfield')
from skyfield.positionlib import ICRF, Barycentric
from skyfield.toposlib import Topos

# Utility
import os

# Plotting
import matplotlib.pyplot as plt

# MSE imports
import kepler_sieve
import astro_utils
from astro_utils import jd_to_mjd, mjd_to_jd
from ra_dec import radec2dir, dir2radec, site2geoloc, qv2dir, direction_diff, radec_diff, skyfield_observe
from horizons_files import load_pos_jpl, load_ast_pos_jpl, load_obs_jpl, load_ast_obs_jpl
from horizons_files import obs_add_interp_qv, obs_ast_add_interp_qv, obs_add_calc_dir, obs_add_radec, obs_direction_diff
from asteroid_element import load_ast_elt
from asteroid_data import make_data_one_file, get_earth_pos
from utils import range_inc

### Observation of Earth and Mars according to JPL

In [2]:
# Data directories
dir_name = '../data/jpl/testing/hourly'
dir_name_daily = '../data/jpl/testing/daily'

# Build DataFrame for earth and mars position at 3 hour frequency
df_earth = load_pos_jpl(body_name='earth', dir_name=dir_name)
df_mars = load_pos_jpl(body_name='mars', dir_name=dir_name)

# Earth at daily frequency
df_earth_daily = load_pos_jpl(body_name='earth', dir_name=dir_name_daily)
# df_mars_daily = load_pos_jpl(body_name='mars', dir_name=dir_name_daily)

### Vectors of First 16 Asteroids from JPL

In [3]:
# Load the asteroid position and velocity from JPL
df_ast = load_ast_pos_jpl(ast_num0=1, ast_num1=16, dir_name=dir_name)
df_ast_daily = load_ast_pos_jpl(ast_num0=1, ast_num1=16, dir_name=dir_name_daily)

### Observations of First 16 Asteroids from JPL

In [4]:
# Load the asteroid observations from JPL
df_obs = load_ast_obs_jpl(ast_num0=1, ast_num1=16, observer_name='palomar', dir_name=dir_name)
df_obs_daily = load_ast_obs_jpl(ast_num0=1, ast_num1=16, observer_name='geocenter', dir_name=dir_name_daily)

### Calculate Asteroid Direction with qv2dir() and MSE Position / Velocity

In [5]:
ast_elt = load_ast_elt()
ast_elt.rename(mapper={'Num':'asteroid_num'}, axis='columns', inplace=True)

In [6]:
ast_elt

,asteroid_num,Name,epoch,a,e,inc,Omega,omega,M,H,G,Ref,f,P,n,long,theta,pomega,T_peri
Num,,,,,,,,,,,,,,,,,,,
1,1,Ceres,58600.0,2.769165,0.076009,0.184901,1.401596,1.284522,1.350398,3.34,0.12,JPL 46,1.501306,1683.145749,0.003733,4.036516,4.187424,2.686118,-361.745873
2,2,Pallas,58600.0,2.772466,0.230337,0.608007,3.020817,5.411373,1.041946,4.13,0.11,JPL 35,1.490912,1686.155979,0.003726,3.190951,3.639917,2.149005,-279.616804
3,3,Juno,58600.0,2.669150,0.256942,0.226699,2.964490,4.330836,0.609557,5.33,0.32,JPL 108,0.996719,1592.787270,0.003945,1.621697,2.008860,1.012141,-154.522558
4,4,Vesta,58600.0,2.361418,0.088721,0.124647,1.811840,2.630709,1.673106,3.20,0.32,JPL 34,-4.436417,1325.432768,0.004740,6.115656,0.006132,4.442550,-352.940421
5,5,Astraea,58600.0,2.574249,0.191095,0.093672,2.470978,6.260280,4.928221,6.85,0.15,JPL 108,-1.738676,1508.600442,0.004165,1.093108,0.709396,2.448072,325.328481
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1255499,1255499,2019 QG,58600.0,0.822197,0.237862,0.220677,5.066979,3.770460,0.503214,21.55,0.15,JPL 1,0.807024,272.309165,0.023074,-3.225717,-2.921908,-3.728932,-21.808984
1255501,1255501,2019 QL,58600.0,2.722045,0.530676,0.113833,4.741919,2.351059,5.297173,19.21,0.15,JPL 1,-2.082964,1640.368337,0.003830,-0.176219,-1.273172,0.809793,257.420824
1255502,1255502,2019 QQ,58600.0,1.053137,0.389091,0.172121,5.648270,2.028352,3.266522,25.31,0.15,JPL 1,-3.081905,394.753268,0.015917,-1.623227,-1.688469,1.393436,189.527723


In [7]:
# Range of asteroids to for data
ast_num_file_start: int = 1
ast_num_file_end: int = 1000
inputs, outputs = make_data_one_file(0, ast_num_file_end)

In [8]:
# The block of asteroid numbers to test (inclusive boundaries)
ast_num_min = 1
ast_num_max = 16

# The number of asteroids, times, and total rows we want to match
# Use daily data to match data file
N_ast = ast_num_max - ast_num_min + 1
N_t = df_earth_daily.mjd.size
N_row = N_ast * N_t
obstime_mjd = df_ast_daily.mjd.values

# Report data shape
print(f'Shape of data frames df_ast and df_obs:')
print(f'N_ast = {N_ast:5} asteroids')
print(f'N_t   = {N_t:5} observation times')
print(f'N_row = {N_row:5} rows in df_ast and df_obs')

Shape of data frames df_ast and df_obs:
N_ast =    16 asteroids
N_t   =  3653 observation times
N_row = 58448 rows in df_ast and df_obs


In [9]:
# Filter for asteroid numbers 
ast_num_file = np.arange(ast_num_file_start, ast_num_file_end, dtype=np.int64)
mask_ast = (ast_num_min <= ast_num_file) & (ast_num_file <= ast_num_max)

# MSE integrated times as one array
ts = inputs['ts'][0]

# Time range for JPL data
t_min = np.min(obstime_mjd)
t_max = np.max(obstime_mjd)

# Filter for MSE times that match
mask_t = (t_min <= ts) & (ts <= t_max)

In [10]:
# Block of asteroid data 
q_ast_all = outputs['q']
v_ast_all = outputs['v']
u_ast_all = outputs['u']

# filter for selected asteroids only
q_ast_all_t = q_ast_all[mask_ast, :, :]
v_ast_all_t = v_ast_all[mask_ast, :, :]
u_ast_all_t = u_ast_all[mask_ast, :, :]

# filter for selected times only
q_ast_mse_3d = q_ast_all_t[:, mask_t, :]
v_ast_mse_3d = v_ast_all_t[:, mask_t, :]
u_ast_mse_3d = u_ast_all_t[:, mask_t, :]

# for some reason i don't understand, can't do these at once
# q_ast_mse = q_ast_all[mask_ast, mask_t, :]
q_ast_mse_3d.shape

(16, 3653, 3)

In [11]:
# Reshape MSE asteroid data to match shape of DataFrame
q_ast_mse = np.zeros((3, N_row))
v_ast_mse = np.zeros((3, N_row))
u_ast_mse = np.zeros((3, N_row))

In [12]:
# Position
q_ast_mse[0, :] = q_ast_mse_3d[:, :, 0].reshape((-1))
q_ast_mse[1, :] = q_ast_mse_3d[:, :, 1].reshape((-1))
q_ast_mse[2, :] = q_ast_mse_3d[:, :, 2].reshape((-1))

# Velocity
v_ast_mse[0, :] = v_ast_mse_3d[:, :, 0].reshape((-1))
v_ast_mse[1, :] = v_ast_mse_3d[:, :, 1].reshape((-1))
v_ast_mse[2, :] = v_ast_mse_3d[:, :, 2].reshape((-1))

# Direction from geocenter
u_ast_mse[0, :] = u_ast_mse_3d[:, :, 0].reshape((-1))
u_ast_mse[1, :] = u_ast_mse_3d[:, :, 1].reshape((-1))
u_ast_mse[2, :] = u_ast_mse_3d[:, :, 2].reshape((-1))

In [13]:
# Compute direction from MSE position and velocity
u_mse = u_ast_mse

# extract u_jpl from df_obs_daily
u_jpl = df_obs_daily[['ux_jpl', 'uy_jpl', 'uz_jpl']].values.T

# difference in directions as a vector
u_diff = u_mse - u_jpl

# norm of difference, converted to arc seconds
u_diff_norm = np.linalg.norm(u_diff, axis=0)
angle_diff = np.rad2deg(u_diff_norm)*3600

# mean error in arc-seconds
mean_error = np.mean(angle_diff)
print(f'mean error: {mean_error:8.3f} arc seconds')

mean error:    0.974 arc seconds
